### Formulierung und Lösung des Produktionsoptimierungsbeispiel mit Gurobi

In [12]:
from gurobipy import *
import numpy as np

Formulierung in mathematischer Form

\begin{alignat}{5}
\max_{x_1, x_2} & \quad  &   2x_1+2x_2 & & & \\[2mm]
\text{s.t. } & &  5x_1+10x_2&\leq 50\\
             & &  12x_1+8x_2&\leq 72\\
             & &  4x_1+0x_2&\leq 20\\
             & &  x_1+x_2&\leq 30\\
             & & x_1, x_2 &\geq 0
\end{alignat}



Zuerst erzeugen wir ein leeres Modell:

In [13]:
m = Model("Production Optimization")

Danach können wir einzelne Variablen anlegen. Als Parameter geben wir an
* Den Namen mit "name="
* Den Variablentyp mit "vtype=". Es gibt folgende Möglichkeiten
  * GRB.CONTINUOUS 
  * GRB.BINARY
  * GRB.INTEGER
* Die untere Schranke für die Variable mit "lb=". Falls nichts angegeben wird, ist die Variable automatisch nichtnegativ. Falls die Variable nach unten unbeschränkt sein soll, können wir "lb=-np.infty" spezifizieren.

In [14]:
x1 = m.addVar(name="x1", vtype=GRB.CONTINUOUS, lb=0)
x2 = m.addVar(name="x2", vtype=GRB.CONTINUOUS, lb=0)

Einzelne Nebenbedinungen können wir dann direkt mit dem Befehl addConstr angeben.

In [15]:
m.addConstr(5*x1+10*x2 <= 50)
m.addConstr(12*x1+8*x2 <= 72)
m.addConstr(4*x1 <= 20)
m.addConstr(x1+x2 <= 30)
#m.addConstr(x1 >= 0) # nicht nötig, da bereits über den Parameter lb=0 beim Anlegen der Variable spezifiziert
#m.addConstr(x2 >= 0)

<gurobi.Constr *Awaiting Model Update*>

Danach folgt die Zielfunktion

In [16]:
m.setObjective(2*x1+2*x2,GRB.MAXIMIZE)

Nun starten wir den Solver

In [17]:
m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4 rows, 2 columns and 7 nonzeros
Model fingerprint: 0xfa36184a
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 7e+01]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+01   8.000000e+00   0.000000e+00      0s
       2    1.4000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.400000000e+01


Die Lösung können wir wie folgt auslesen:

In [20]:
print(f"x1={x1.x}, x2={x2.x}")

x1=4.0, x2=3.0


Bei Bedarf kann man sich das Model über den Befehl display ausgeben lassen.

In [19]:
m.display()

Maximize
  <gurobi.LinExpr: 2.0 x1 + 2.0 x2>
Subject To
  R0: <gurobi.LinExpr: 5.0 x1 + 10.0 x2> <= 50
  R1: <gurobi.LinExpr: 12.0 x1 + 8.0 x2> <= 72
  R2: <gurobi.LinExpr: 4.0 x1> <= 20
  R3: <gurobi.LinExpr: x1 + x2> <= 30
